# Imports

In [1]:
#general imports
import pandas as pd
import numpy as np

#statsmodels for regression
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col

#scipy for testing
from scipy import stats

#for visualization
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

# Weather Data

In [2]:
# csv import of weather data
df_weather = pd.read_csv('../data/weather_hourly_philadelphia.csv')

In [3]:
# change type of "date_time" to datetime
df_weather["date_time"] = pd.to_datetime(df_weather["date_time"])

In [4]:
# ordered weather data by date
df_weather = df_weather.sort_values(by=['date_time'])

In [5]:
# deleted weather date which is not needed for Philadelphia 2017

start2017 = datetime(2016, 12, 31, 23)
end2017 = datetime(2018, 1, 1)

df_2017weather = df_weather[(df_weather["date_time"] > start2017) & (df_weather["date_time"] < end2017)]
df_2017weather_unique = df_2017weather.drop_duplicates(subset='date_time')
df_2017weather_unique['Order'] = np.arange(len(df_2017weather_unique))
df_2017weather_unique = df_2017weather_unique.set_index('Order')

df_2017weather_unique.tail(50)

<ipython-input-5-2553c7c29634>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2017weather_unique['Order'] = np.arange(len(df_2017weather_unique))


,date_time,max_temp,min_temp,precip
Order,,,,
8182,2017-12-29 21:00:00,-4.4,-4.4,0.0
8183,2017-12-29 22:00:00,-5.0,-5.0,0.0
8184,2017-12-29 23:00:00,-5.6,-5.6,0.0
8185,2017-12-30 00:00:00,-5.6,-5.6,0.0
8186,2017-12-30 01:00:00,-6.1,-6.1,0.0
8187,2017-12-30 02:00:00,-6.7,-6.7,0.0
8188,2017-12-30 03:00:00,-7.2,-7.2,0.0
8189,2017-12-30 04:00:00,-7.2,-7.2,0.0
8190,2017-12-30 05:00:00,-7.8,-7.8,0.0


In [6]:
df_2017weather_unique['date_time'][1] - df_2017weather_unique['date_time'][1]

Timedelta('0 days 00:00:00')

In [7]:
for i in df_2017weather_unique.index:
    if i == 8231:
        print("End")
    elif (df_2017weather_unique['date_time'][i] - df_2017weather_unique['date_time'][i+1]) == (df_2017weather_unique['date_time'][1] - df_2017weather_unique['date_time'][1]):
        df_2017weather_support = {'date_time': df_2017weather_unique['date_time'][i], 'max_temp': df_2017weather_unique['max_temp'][i], 'min_temp': df_2017weather_unique['max_temp'][i], 'precip': df_2017weather_unique['precip'][i]}
        print(df_2017weather_support)


End


In [8]:
# identify missing data and generate by using average data from former and following hour

for i in df_2017weather_unique.index:
    if i == 8231:
        print("End")
    elif (df_2017weather_unique['date_time'][i] - df_2017weather_unique['date_time'][i+1]) != (df_2017weather_unique['date_time'][1] - df_2017weather_unique['date_time'][2]):
        df_2017weather_support = {'date_time': (df_2017weather_unique['date_time'][i] + pd.DateOffset(hours=1)), 'max_temp': (df_2017weather_unique['max_temp'][i] + df_2017weather_unique['max_temp'][i+1])/2, 'min_temp': (df_2017weather_unique['max_temp'][i] + df_2017weather_unique['max_temp'][i+1])/2, 'precip': df_2017weather_unique['precip'][i]}
        print(df_2017weather_support)
        df_2017weather_unique = df_2017weather_unique.append(df_2017weather_support, ignore_index=True)

{'date_time': Timestamp('2017-01-02 15:00:00'), 'max_temp': 5.0, 'min_temp': 5.0, 'precip': 1.0}
{'date_time': Timestamp('2017-01-02 17:00:00'), 'max_temp': 5.85, 'min_temp': 5.85, 'precip': 1.0}
{'date_time': Timestamp('2017-01-02 23:00:00'), 'max_temp': 5.3, 'min_temp': 5.3, 'precip': 0.0}
{'date_time': Timestamp('2017-01-03 02:00:00'), 'max_temp': 5.0, 'min_temp': 5.0, 'precip': 0.0}
{'date_time': Timestamp('2017-01-03 05:00:00'), 'max_temp': 5.0, 'min_temp': 5.0, 'precip': 0.0}
{'date_time': Timestamp('2017-01-03 13:00:00'), 'max_temp': 5.3, 'min_temp': 5.3, 'precip': 1.0}
{'date_time': Timestamp('2017-01-03 22:00:00'), 'max_temp': 8.05, 'min_temp': 8.05, 'precip': 0.0}
{'date_time': Timestamp('2017-01-04 03:00:00'), 'max_temp': 6.7, 'min_temp': 6.7, 'precip': 0.0}
{'date_time': Timestamp('2017-01-04 07:00:00'), 'max_temp': 6.7, 'min_temp': 6.7, 'precip': 1.0}
{'date_time': Timestamp('2017-01-04 14:00:00'), 'max_temp': 8.3, 'min_temp': 8.3, 'precip': 0.0}
{'date_time': Timestamp('2

In [9]:
# new order by date

df_2017weather_unique = df_2017weather_unique.sort_values(by=['date_time'])
df_2017weather_unique['Order'] = np.arange(len(df_2017weather_unique))
df_2017weather_unique = df_2017weather_unique.set_index('Order')

df_2017weather_unique.tail(20)

,date_time,max_temp,min_temp,precip
Order,,,,
8739,2017-12-31 04:00:00,-6.1,-6.1,0.0
8740,2017-12-31 05:00:00,-6.7,-6.7,0.0
8741,2017-12-31 06:00:00,-6.7,-6.7,0.0
8742,2017-12-31 07:00:00,-7.2,-7.2,0.0
8743,2017-12-31 08:00:00,-7.8,-7.8,0.0
8744,2017-12-31 09:00:00,-8.3,-8.3,0.0
8745,2017-12-31 10:00:00,-8.3,-8.3,0.0
8746,2017-12-31 11:00:00,-8.9,-8.9,0.0
8747,2017-12-31 12:00:00,-9.4,-9.4,0.0


In [10]:
# completing data

for i in df_2017weather_unique.index:
    if i == 8758:
        print("End")
    elif (df_2017weather_unique['date_time'][i] - df_2017weather_unique['date_time'][i+1]) != (df_2017weather_unique['date_time'][1] - df_2017weather_unique['date_time'][2]):
        df_2017weather_support = {'date_time': (df_2017weather_unique['date_time'][i] + pd.DateOffset(hours=1)), 'max_temp': (df_2017weather_unique['max_temp'][i] + df_2017weather_unique['max_temp'][i+1])/2, 'min_temp': (df_2017weather_unique['max_temp'][i] + df_2017weather_unique['max_temp'][i+1])/2, 'precip': df_2017weather_unique['precip'][i]}
        print(df_2017weather_support)
        df_2017weather_unique = df_2017weather_unique.append(df_2017weather_support, ignore_index=True)

{'date_time': Timestamp('2017-09-17 05:00:00'), 'max_temp': 21.375, 'min_temp': 21.375, 'precip': 0.0}
End


In [11]:
# new order by date

df_2017weather_unique = df_2017weather_unique.sort_values(by=['date_time'])
df_2017weather_unique['Order'] = np.arange(len(df_2017weather_unique))
df_2017weather_unique = df_2017weather_unique.set_index('Order')

df_2017weather_unique.tail(20)

,date_time,max_temp,min_temp,precip
Order,,,,
8740,2017-12-31 04:00:00,-6.1,-6.1,0.0
8741,2017-12-31 05:00:00,-6.7,-6.7,0.0
8742,2017-12-31 06:00:00,-6.7,-6.7,0.0
8743,2017-12-31 07:00:00,-7.2,-7.2,0.0
8744,2017-12-31 08:00:00,-7.8,-7.8,0.0
8745,2017-12-31 09:00:00,-8.3,-8.3,0.0
8746,2017-12-31 10:00:00,-8.3,-8.3,0.0
8747,2017-12-31 11:00:00,-8.9,-8.9,0.0
8748,2017-12-31 12:00:00,-9.4,-9.4,0.0


In [12]:
# check if data is complete

for i in df_2017weather_unique.index:
    if i == 8231:
        print("Data complete")
    elif (df_2017weather_unique['date_time'][i] - df_2017weather_unique['date_time'][i+1]) != (df_2017weather_unique['date_time'][1] - df_2017weather_unique['date_time'][2]):
        df_2017weather_support = {'date_time': (df_2017weather_unique['date_time'][i] + pd.DateOffset(hours=1)), 'max_temp': (df_2017weather_unique['max_temp'][i] + df_2017weather_unique['max_temp'][i+1])/2, 'min_temp': (df_2017weather_unique['max_temp'][i] + df_2017weather_unique['max_temp'][i+1])/2, 'precip': df_2017weather_unique['precip'][i]}
        print(df_2017weather_support)
        df_2017weather_unique = df_2017weather_unique.append(df_2017weather_support, ignore_index=True)

Data complete


KeyError: 8760

In [ ]:
df_2017weather_unique